In [162]:
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [108]:
# Buscamos que la url sea como nosotros la pedimos

date_1 = '2024-10-25'
date_2 = '2024-10-27'
ciudad = 'valencia'
pagina = 1

url = f"https://www.civitatis.com/es/madrid/?page={pagina}&fromDate={date_1}&toDate={date_2}"

In [136]:
#Abrimos en navegador con la Url introducida
driver = webdriver.Chrome()
url_wunder = url

driver.get(url_wunder)
driver.maximize_window()
sleep(5)
driver.close()

In [167]:
# Comprobamos que la request sea corrrecta

url_civitatis = "https://www.civitatis.com/es/madrid/?page={pagina}&fromDate={date_1}&toDate={date_2}"
res_civitatis = requests.get(url)
print(f"La respuesta de la petición es: {res_civitatis.status_code}")

La respuesta de la petición es: 200


In [117]:
#creamos nuestra sopa 

sopa_civitatis = BeautifulSoup(res_civitatis.content, "html.parser")

In [118]:
# comprobamo snuestra sopa
sopa_civitatis

<!DOCTYPE html>

<html data-lang="es" lang="es">
<head>
<script type="text/javascript">window.gdprAppliesGlobally=true;(function(){function n(e){if(!window.frames[e]){if(document.body&&document.body.firstChild){var t=document.body;var r=document.createElement("iframe");r.style.display="none";r.name=e;r.title=e;t.insertBefore(r,t.firstChild)}else{setTimeout(function(){n(e)},5)}}}function e(r,a,o,s,c){function e(e,t,r,n){if(typeof r!=="function"){return}if(!window[a]){window[a]=[]}var i=false;if(c){i=c(e,n,r)}if(!i){window[a].push({command:e,version:t,callback:r,parameter:n})}}e.stub=true;e.stubVersion=2;function t(n){if(!window[r]||window[r].stub!==true){return}if(!n.data){return}var i=typeof n.data==="string";var e;try{e=i?JSON.parse(n.data):n.data}catch(t){return}if(e[o]){var a=e[o];window[r](a.command,a.version,function(e,t){var r={};r[s]={returnValue:e,success:t,callId:a.callId};n.source.postMessage(i?JSON.stringify(r):r,"*")},a.parameter)}}if(typeof window[r]!=="function"){window[r

In [160]:
# Creamos nuestro bucle para iterar de los datos scrapeados y coger lo que no shace falta

nombres_actividades = []
descripciones_actividades = []
precios_actividades = []

bloque_total = sopa_civitatis.findAll("div", {"class": "o-search-list__item"} )
for i in bloque_total:
    nombre_actividad = i.find("h2").text.strip()
    descripcion_actividad = i.find("div",{"class":"comfort-card__text l-list-card__text"}).text.strip()
    precio_actividad = i.find("span",{"class":"comfort-card__price__text"}).text.strip()

# Lo añadimos a la lista de listas
    nombres_actividades.append(nombre_actividad)
    descripciones_actividades.append(descripcion_actividad)
    precios_actividades.append(precio_actividad)

# Creamos un documento
datos = {
    'Nombre Actividad': nombres_actividades,
    'Descripción Actividad': descripciones_actividades,
    'Precio Actividad': precios_actividades
}

# Creamos los datos de diccionario en un DataFrame
df = pd.DataFrame(datos)
df

,Nombre Actividad,Descripción Actividad,Precio Actividad
0,Visita guiada por el Palacio Real,En esta visita guiada descubriremos la histori...,32 €
1,Excursión a Toledo y Segovia,En este tour visitaremos las dos ciudades más ...,59 €
2,Tour Bernabéu,En la visita oficial del Estadio Santiago Bern...,35 €
3,Free tour por Madrid,El free tour de Civitatis es la mejor forma de...,¡Gratis!
4,Excursión a Toledo de día completo con entradas,En esta excursión a Toledo disfrutaremos de lo...,"81,60 €"
5,Visita guiada por el Museo del Prado,En esta visita guiada por el Museo del Prado r...,36 €
6,Traslados en Madrid,"Servicio privado, puerta a puerta y al mejor p...",26 €
7,Espectáculo flamenco en Torres Bermejas,Vive el arte del flamenco en el tablao Torres ...,28 €
8,Excursión a Ávila y Segovia,En esta excursión a Ávila y Segovia visitaremo...,"68,40 €"
9,Visita guiada por el Museo del Prado y el Pala...,En esta visita guiada descubriremos las obras ...,68 €
